In [19]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, current_timestamp, date_format, rand, expr
from pyspark.sql.functions import to_timestamp, dayofyear, date_format, sum, min, max, count, desc
from pyspark.sql.functions import to_date, col

# Initializing Spark session
spark = SparkSession.builder \
    .appName("Generate Semi-Structured Logs") \
    .getOrCreate()

# Generating semi-structured log data
num_logs = 1000000

logs_df = spark.range(0, num_logs) \
    .withColumn('timestamp', current_timestamp()) \
    .withColumn('remote_host', expr("concat_ws('.', floor(rand() * 256), floor(rand() * 256), floor(rand() * 256), floor(rand() * 256))")) \
    .withColumn('request_method', expr("CASE WHEN rand() < 0.25 THEN 'GET' WHEN rand() < 0.5 THEN 'POST' WHEN rand() < 0.75 THEN 'PUT' ELSE 'DELETE' END")) \
    .withColumn('request_endpoint', expr("concat('/api/', substring('abcdefghijklmnopqrstuvwxyz0123456789', floor(rand() * 25) + 1, 10))")) \
    .withColumn('protocol', expr("CASE WHEN rand() < 0.5 THEN 'HTTP/1.1' ELSE 'HTTP/2.0' END")) \
    .withColumn('status_code', expr("CASE WHEN rand() < 0.8 THEN 200 WHEN rand() < 0.9 THEN 404 ELSE 500 END")) \
    .withColumn('content_size', expr("floor(rand() * 10000)"))

# Saving the DataFrame as a Parquet file
logs_df.write.mode('overwrite').parquet('semi_structured_logs.parquet')


In [ ]:
# Loading semi-structured logs data
logs_df = spark.read.parquet("semi_structured_logs.parquet")

# Displaying the DataFrame
logs_df.show(10, truncate=False)

+---+--------------------------+---------------+--------------+----------------+--------+-----------+------------+
|id |timestamp                 |remote_host    |request_method|request_endpoint|protocol|status_code|content_size|
+---+--------------------------+---------------+--------------+----------------+--------+-----------+------------+
|0  |2024-05-22 23:26:46.296121|209.90.27.188  |GET           |/api/lmnopqrstu |HTTP/2.0|500        |9128        |
|1  |2024-05-22 23:26:46.296121|253.74.76.20   |POST          |/api/ijklmnopqr |HTTP/2.0|200        |4981        |
|2  |2024-05-22 23:26:46.296121|191.137.126.135|POST          |/api/opqrstuvwx |HTTP/2.0|200        |1848        |
|3  |2024-05-22 23:26:46.296121|214.10.186.118 |DELETE        |/api/pqrstuvwxy |HTTP/1.1|200        |6535        |
|4  |2024-05-22 23:26:46.296121|57.143.11.191  |PUT           |/api/xyz0123456 |HTTP/2.0|404        |3692        |
|5  |2024-05-22 23:26:46.296121|161.45.165.178 |PUT           |/api/vwxyz01234 |

In [ ]:
# Converting and extracting timestamp information
logs_df = logs_df.withColumn('timestamp', to_timestamp('timestamp'))
logs_df = logs_df.withColumn('day', dayofyear('timestamp'))
logs_df = logs_df.withColumn('date', date_format('timestamp', 'yyyy-MM-dd'))
logs_df.select('timestamp', 'day', 'date').show(10, truncate=False)

+--------------------------+---+----------+
|timestamp                 |day|date      |
+--------------------------+---+----------+
|2024-05-22 23:26:46.296121|143|2024-05-22|
|2024-05-22 23:26:46.296121|143|2024-05-22|
|2024-05-22 23:26:46.296121|143|2024-05-22|
|2024-05-22 23:26:46.296121|143|2024-05-22|
|2024-05-22 23:26:46.296121|143|2024-05-22|
|2024-05-22 23:26:46.296121|143|2024-05-22|
|2024-05-22 23:26:46.296121|143|2024-05-22|
|2024-05-22 23:26:46.296121|143|2024-05-22|
|2024-05-22 23:26:46.296121|143|2024-05-22|
|2024-05-22 23:26:46.296121|143|2024-05-22|
+--------------------------+---+----------+
only showing top 10 rows



In [ ]:
# Calculating statistics related to content size
# Top endpoints content
top_endpoints = logs_df.groupBy("request_endpoint").count().orderBy("count", ascending=False)
top_endpoints.show(10)

+----------------+-----+
|request_endpoint|count|
+----------------+-----+
| /api/yz01234567|40290|
| /api/mnopqrstuv|40207|
| /api/uvwxyz0123|40186|
| /api/lmnopqrstu|40184|
| /api/abcdefghij|40162|
| /api/defghijklm|40138|
| /api/opqrstuvwx|40108|
| /api/vwxyz01234|40092|
| /api/qrstuvwxyz|40085|
| /api/cdefghijkl|40081|
+----------------+-----+
only showing top 10 rows



In [ ]:
# Top endpoints transferring maximum content
top_endpoints_by_content = logs_df.groupBy("request_endpoint").sum("content_size") \
    .orderBy(desc("sum(content_size)")).withColumnRenamed("sum(content_size)", "total_content_size")
top_endpoints_by_content.show(10)

+----------------+------------------+
|request_endpoint|total_content_size|
+----------------+------------------+
| /api/yz01234567|         202089420|
| /api/uvwxyz0123|         201605100|
| /api/abcdefghij|         201210681|
| /api/xyz0123456|         200683018|
| /api/defghijklm|         200682981|
| /api/cdefghijkl|         200632905|
| /api/opqrstuvwx|         200542859|
| /api/lmnopqrstu|         200385632|
| /api/mnopqrstuv|         200348891|
| /api/qrstuvwxyz|         200214378|
+----------------+------------------+
only showing top 10 rows



In [ ]:
# Daily visited content size
daily_content_size = logs_df.withColumn("date", to_date(col("timestamp"))).groupBy("date").agg(sum("content_size").alias("daily_content_size")).orderBy("date")
daily_content_size.show()

+----------+------------------+
|      date|daily_content_size|
+----------+------------------+
|2024-05-22|        4999257546|
+----------+------------------+



In [ ]:
# Min, Max and Count of content size
content_size_stats = logs_df.agg(min('content_size').alias('min_size'), max('content_size').alias('max_size'), count('content_size').alias('count_size'))
content_size_stats.show()

+--------+--------+----------+
|min_size|max_size|count_size|
+--------+--------+----------+
|       0|    9999|   1000000|
+--------+--------+----------+



In [ ]:
# Response Code Analysis
response_code_analysis = logs_df.groupBy('status_code').count().orderBy('count', ascending=False)
response_code_analysis.show()

+-----------+------+
|status_code| count|
+-----------+------+
|        200|799942|
|        404|180219|
|        500| 19839|
+-----------+------+



In [ ]:
# Frequent Visitors
frequent_visitors = logs_df.groupBy("remote_host").count().filter(col("count") >= 1).orderBy("count", ascending=False)
frequent_visitors.show(10)

+---------------+-----+
|    remote_host|count|
+---------------+-----+
| 100.178.210.52|    2|
|  196.187.29.40|    2|
|  86.74.213.236|    2|
|235.205.182.224|    2|
|166.180.218.170|    2|
|   211.3.27.154|    2|
|  58.210.249.35|    2|
|106.133.195.108|    2|
|  142.85.90.107|    2|
| 114.87.122.139|    2|
+---------------+-----+
only showing top 10 rows



In [ ]:
# Identifying IP addresses accessing the server more than 10 times
frequent_ip_addresses = logs_df.groupBy('remote_host').agg(count('*').alias('access_count')).filter('access_count > 10').orderBy(desc('access_count'))
frequent_ip_addresses.show()

+-----------+------------+
|remote_host|access_count|
+-----------+------------+
+-----------+------------+



In [ ]:
# Analyzing bad requests including the top 10 latest 404 requests with their endpoints and time
latest_404_requests = logs_df.filter(col("status_code") == 404).orderBy(col("timestamp"),\
                       ascending=False).select("timestamp", "request_endpoint").limit(10)
latest_404_requests.show(truncate=False)

+--------------------------+----------------+
|timestamp                 |request_endpoint|
+--------------------------+----------------+
|2024-05-22 23:26:46.296121|/api/mnopqrstuv |
|2024-05-22 23:26:46.296121|/api/xyz0123456 |
|2024-05-22 23:26:46.296121|/api/vwxyz01234 |
|2024-05-22 23:26:46.296121|/api/jklmnopqrs |
|2024-05-22 23:26:46.296121|/api/yz01234567 |
|2024-05-22 23:26:46.296121|/api/pqrstuvwxy |
|2024-05-22 23:26:46.296121|/api/opqrstuvwx |
|2024-05-22 23:26:46.296121|/api/defghijklm |
|2024-05-22 23:26:46.296121|/api/rstuvwxyz0 |
|2024-05-22 23:26:46.296121|/api/jklmnopqrs |
+--------------------------+----------------+



In [ ]:
# Calculating percentage of successful requests
total_requests = logs_df.count()
successful_requests = logs_df.filter(col("status_code") == 200).count()
error_requests = total_requests - successful_requests

# Calculating success and error percentages
success_percentage = (successful_requests / total_requests) * 100
error_percentage = 100 - success_percentage

# Defining thresholds for anomaly detection
success_threshold = 95  # Threshold for successful requests percentage
error_threshold = 5  # Threshold for error requests percentage

# Transformation for anomaly detection
def detect_anomalies(success_percentage, error_percentage):
    if success_percentage < success_threshold:
        return "Anomaly: Low success rate detected"
    elif error_percentage > error_threshold:
        return "Anomaly: High error rate detected"
    else:
        return "No anomalies detected"

# Applying transformation to detect anomalies
anomaly_detection_result = detect_anomalies(success_percentage, error_percentage)

# Printing analysis results and anomaly detection
print(f"Successful Requests: {success_percentage:.2f}%")
print(f"Error Requests: {error_percentage:.2f}%")
print(anomaly_detection_result)

Successful Requests: 79.99%
Error Requests: 20.01%
Anomaly: Low success rate detected
